In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Define the regions
regions = ['PJM', 'NEPOOL', 'NYISO', 'MISO', 'ERCOT']

# Generate hourly timestamps for 10 years
end_date = datetime.now()
start_date = end_date - timedelta(days=10*365)
timestamps = pd.date_range(start=start_date, end=end_date, freq='H')

# Create a sample dataframe
data = []
for region in regions:
    temperatures = np.random.randint(-20, 40, size=len(timestamps))  # Random temperatures between -20 and 40
    for i, timestamp in enumerate(timestamps):
        data.append([timestamp, region, temperatures[i]])

df = pd.DataFrame(data, columns=['Timestamp', 'Region', 'Temperature'])
df.head()

In [ ]:
# Reshape the dataframe
reshaped_df = df.pivot(index='Timestamp', columns='Region', values='Temperature')
reshaped_df.reset_index(inplace=True)
reshaped_df.head()

In [ ]:
import matplotlib.pyplot as plt

def plot_avg_temperature(month):
    # Filter the dataframe for the given month
    month_df = reshaped_df[reshaped_df['Timestamp'].dt.month == month]

    # Calculate the average temperature for each year
    avg_temperatures = month_df.groupby(month_df['Timestamp'].dt.year).mean().drop(columns='Timestamp')

    # Plot the average temperatures
    plt.figure(figsize=(12, 8))
    for region in regions:
        plt.plot(avg_temperatures.index, avg_temperatures[region], label=region, marker='o')

    plt.title(f'Average Temperature for Month {month} Across Years')
    plt.xlabel('Year')
    plt.ylabel('Average Temperature')
    plt.legend()
    plt.grid(True)
    plt.show()

# Test the function for month 1 (January)
plot_avg_temperature(1)

KeyError: "['Timestamp'] not found in axis"

In [ ]:
def plot_avg_temperature(month):
    # Filter the dataframe for the given month
    month_df = reshaped_df[reshaped_df['Timestamp'].dt.month == month]

    # Calculate the average temperature for each year
    avg_temperatures = month_df.groupby(month_df['Timestamp'].dt.year).mean()

    # Plot the average temperatures
    plt.figure(figsize=(12, 8))
    for region in regions:
        plt.plot(avg_temperatures.index, avg_temperatures[region], label=region, marker='o')

    plt.title(f'Average Temperature for Month {month} Across Years')
    plt.xlabel('Year')
    plt.ylabel('Average Temperature')
    plt.legend()
    plt.grid(True)
    plt.show()

# Test the function for month 1 (January)
plot_avg_temperature(1)

In [ ]:
def plot_avg_temperature_v2(month):
    # Filter the dataframe for the given month
    month_df = reshaped_df[reshaped_df['Timestamp'].dt.month == month]

    # Calculate the average temperature for each year
    avg_temperatures = month_df.groupby(month_df['Timestamp'].dt.year).mean()

    plt.figure(figsize=(15, 10))

    for region in regions:
        sorted_avg_temps = avg_temperatures[region].sort_values()
        years = sorted_avg_temps.index
        temps = sorted_avg_temps.values
        plt.plot(temps, range(len(temps)), label=region, marker='o')
        for i, (year, temp) in enumerate(zip(years, temps)):
            plt.annotate(f'{year}', (temp, i), textcoords="offset points", xytext=(0,10), ha='center')
            plt.annotate(f'{temp:.2f}°', (temp, i), textcoords="offset points", xytext=(0,-15), ha='center')

    plt.title(f'Average Temperature for Month {month} Across Years')
    plt.xlabel('Average Temperature')
    plt.ylabel('Sorted Years (Coldest to Hottest)')
    plt.yticks([])  # Hide y-axis ticks
    plt.legend()
    plt.grid(True, axis='x')
    plt.show()

# Test the function for month 1 (January)
plot_avg_temperature_v2(1)

In [ ]:
def plot_avg_temperature_v3(month):
    # Filter the dataframe for the given month
    month_df = reshaped_df[reshaped_df['Timestamp'].dt.month == month]

    # Calculate the average temperature for each year
    avg_temperatures = month_df.groupby(month_df['Timestamp'].dt.year).mean()

    plt.figure(figsize=(15, 10))

    for idx, region in enumerate(regions):
        sorted_avg_temps = avg_temperatures[region].sort_values()
        years = sorted_avg_temps.index
        temps = sorted_avg_temps.values
        plt.hlines(y=idx, xmin=temps.min(), xmax=temps.max(), color='gray', alpha=0.5)  # Flat line
        for i, (year, temp) in enumerate(zip(years, temps)):
            plt.scatter(temp, idx, marker='o')
            plt.annotate(f'{year}', (temp, idx), textcoords="offset points", xytext=(0,10), ha='center')
            plt.annotate(f'{temp:.2f}°', (temp, idx), textcoords="offset points", xytext=(0,-15), ha='center')

    plt.title(f'Average Temperature for Month {month} Across Years')
    plt.xlabel('Average Temperature')
    plt.yticks(range(len(regions)), regions)
    plt.grid(True, axis='x')
    plt.show()

# Test the function for month 1 (January)
plot_avg_temperature_v3(1)

In [ ]:
def plot_avg_temperature_difference(month):
    # Filter the dataframe for the given month
    month_df = reshaped_df[reshaped_df['Timestamp'].dt.month == month]

    # Calculate the 5-year rolling average temperature for each year
    rolling_avg_temperatures = month_df.groupby(month_df['Timestamp'].dt.year).mean().rolling(window=5).mean()

    # Calculate the difference from the 5-year average
    difference_from_rolling_avg = month_df.groupby(month_df['Timestamp'].dt.year).mean() - rolling_avg_temperatures

    plt.figure(figsize=(15, 10))

    for idx, region in enumerate(regions):
        sorted_diffs = difference_from_rolling_avg[region].sort_values()
        years = sorted_diffs.index
        diffs = sorted_diffs.values
        plt.hlines(y=idx, xmin=diffs.min(), xmax=diffs.max(), color='gray', alpha=0.5)  # Flat line
        for i, (year, diff) in enumerate(zip(years, diffs)):
            if not np.isnan(diff):
                plt.scatter(diff, idx, marker='o')
                plt.annotate(f'{year}', (diff, idx), textcoords="offset points", xytext=(0,10), ha='center')
                plt.annotate(f'{diff:.2f}°', (diff, idx), textcoords="offset points", xytext=(0,-15), ha='center')

    plt.title(f'Difference from 5-Year Average Temperature for Month {month} Across Years')
    plt.xlabel('Temperature Difference from 5-Year Average')
    plt.yticks(range(len(regions)), regions)
    plt.grid(True, axis='x')
    plt.show()

# Test the function for month 1 (January)
plot_avg_temperature_difference(1)

In [ ]:
def plot_avg_temperature_difference_v2(df, month):
    # Filter the dataframe for the given month
    month_df = df[df['Timestamp'].dt.month == month]
    month_name = month_df['Timestamp'].dt.strftime('%B').iloc[0]

    # Calculate the 5-year rolling average temperature for each year
    rolling_avg_temperatures = month_df.groupby(month_df['Timestamp'].dt.year).mean().rolling(window=5).mean()

    # Calculate the difference from the 5-year average
    difference_from_rolling_avg = month_df.groupby(month_df['Timestamp'].dt.year).mean() - rolling_avg_temperatures

    plt.figure(figsize=(15, 10))

    for idx, region in enumerate(regions):
        sorted_diffs = difference_from_rolling_avg[region].sort_values()
        years = sorted_diffs.index
        diffs = sorted_diffs.values
        plt.hlines(y=idx, xmin=diffs.min(), xmax=diffs.max(), color='gray', alpha=0.5)  # Flat line
        for i, (year, diff) in enumerate(zip(years, diffs)):
            if not np.isnan(diff):
                plt.scatter(diff, idx, marker='o')
                plt.annotate(f'{year}', (diff, idx), textcoords="offset points", xytext=(0,10), ha='center')
                plt.annotate(f'{diff:.2f}°', (diff, idx), textcoords="offset points", xytext=(0,-15), ha='center')

    plt.title(f'Difference from 5-Year Average Temperature for {month_name} Across Years')
    plt.xlabel('Temperature Difference from 5-Year Average')
    plt.yticks(range(len(regions)), regions)
    plt.grid(True, axis='x')
    plt.show()

# Test the function for month 1 (January)
plot_avg_temperature_difference_v2(reshaped_df, 1)

In [ ]:
# Filter the dataframe for January
january_df = reshaped_df[reshaped_df['Timestamp'].dt.month == 1]

# Calculate the 5-year rolling average temperature for each year
rolling_avg_temperatures = january_df.groupby(january_df['Timestamp'].dt.year).mean().rolling(window=5).mean()

# Extract the 5-year rolling average for 2021 and 2022
rolling_avg_2021 = rolling_avg_temperatures.loc[2021]
rolling_avg_2022 = rolling_avg_temperatures.loc[2022]

rolling_avg_2021, rolling_avg_2022

(Region
 ERCOT     9.757527
 MISO      9.541935
 NEPOOL    9.446237
 NYISO     9.578495
 PJM       9.164516
 Name: 2021, dtype: float64,
 Region
 ERCOT     9.736022
 MISO      9.656989
 NEPOOL    9.300538
 NYISO     9.437366
 PJM       9.216935
 Name: 2022, dtype: float64)